In [1]:
import numpy as np


In [ ]:
'''
성능 개선이 문제.
list에서 검색은 루프 탐색보다. in 키워드로 검색하는 것이 3배는 빠름.

쓸데없이 리스트에 추가하는 것 삭제. insert 횟수 감소.

위치관계에 따른 그룹을 직접 알기 위해 group matrix 추가


'''

In [2]:

input = [[1, 4, 8, 10], [5, 5, 5, 5], [10, 10, 10, 10], [10, 10, 10, 20]]
height = 3
result=15

# input = [[10, 11, 10, 11], [2, 21, 20, 10], [1, 20, 21, 11], [2, 1, 2, 1]]
# height = 1
# result=18

n = len(input)
npinput = np.asarray(input)

In [3]:
# group api
groups = []

# gi=group index. -1=new
# return ; group index
def group_add(i,j, gi):
    global groups
    if gi==-1:
        groups.append([(i,j)])
        gi = len(groups)-1
        print('newgroup:',gi, 'item=', i,j)
    else:
        groups[gi].append((i,j))
        print('add group=',gi, 'item=', i,j)
    return gi

def group_find(i,j):
    if i<0 or j<0:
        return -1
    if i>=n or j>=n:
        return -1
    for gi, g in enumerate(groups):
        for f in g:
            if f[0]==i and f[1]==j:
                return gi
    return -1

# d = l, r, u, d  (left, right, up, down)
def can_move(i,j, d):
    i2,j2 = i,j
    if d=='l': j2 -=1
    if d=='r': j2 +=1
    if d=='u': i2 -=1
    if d=='d': i2 +=1
    if i2<0 or j2<0:
        return False
    if i2>=n or j2>=n:
        return False
    return abs(input[i][j] - input[i2][j2])<=height

def group_merge(gi, gj):
    global groups
    bak = groups[gj][:]
    print('group merge ',gj,'into',gi)
    groups[gi].extend(bak)
    del groups[gj]
    

In [4]:
groups.clear()
print(group_add(0,0,-1))
print(group_add(1,1,0))
print(group_add(2,2,-1))
print(group_add(3,3,1))
print(group_add(4,4,-1))
print(group_add(5,6,2))

print('find=', group_find(5,6))
print('find=', group_find(2,2))

groups

newgroup: 0 item= 0 0
0
add group= 0 item= 1 1
0
newgroup: 1 item= 2 2
1
add group= 1 item= 3 3
1
newgroup: 2 item= 4 4
2
add group= 2 item= 5 6
2
find= -1
find= 1


[[(0, 0), (1, 1)], [(2, 2), (3, 3)], [(4, 4), (5, 6)]]

In [5]:
groups

[[(0, 0), (1, 1)], [(2, 2), (3, 3)], [(4, 4), (5, 6)]]

In [6]:
group_merge(1,2)

group merge  2 into 1


In [7]:
groups

[[(0, 0), (1, 1)], [(2, 2), (3, 3), (4, 4), (5, 6)]]

In [8]:
can_move(0,2,'r')

True

In [9]:
global groups
print(input)
groups.clear()

for i in range(n):
    for j in range(n):
        gi = group_find(i,j)
        if gi==-1:
            gi = group_add(i,j, -1)
        print('check ', i,j, 'group=', gi)
            
        if can_move(i,j,'r'):
            g2 = group_find(i,j+1)
            if g2==-1:
                group_add(i,j+1,gi)
            elif g2==gi:
                pass
            else:
                group_merge(min(gi,g2), max(gi,g2))
                gi = min(gi, g2)

        if can_move(i,j,'d'):
            g2 = group_find(i+1,j)
            if g2==-1:
                group_add(i+1,j,gi)
            elif g2==gi:
                pass
            else:
                group_merge(min(gi,g2), max(gi,g2))

print(groups)


[[1, 4, 8, 10], [5, 5, 5, 5], [10, 10, 10, 10], [10, 10, 10, 20]]
newgroup: 0 item= 0 0
check  0 0 group= 0
add group= 0 item= 0 1
check  0 1 group= 0
add group= 0 item= 1 1
newgroup: 1 item= 0 2
check  0 2 group= 1
add group= 1 item= 0 3
add group= 1 item= 1 2
check  0 3 group= 1
newgroup: 2 item= 1 0
check  1 0 group= 2
group merge  2 into 0
check  1 1 group= 0
group merge  1 into 0
check  1 2 group= 0
add group= 0 item= 1 3
check  1 3 group= 0
newgroup: 1 item= 2 0
check  2 0 group= 1
add group= 1 item= 2 1
add group= 1 item= 3 0
check  2 1 group= 1
add group= 1 item= 2 2
add group= 1 item= 3 1
check  2 2 group= 1
add group= 1 item= 2 3
add group= 1 item= 3 2
check  2 3 group= 1
check  3 0 group= 1
check  3 1 group= 1
check  3 2 group= 1
newgroup: 2 item= 3 3
check  3 3 group= 2
[[(0, 0), (0, 1), (1, 1), (1, 0), (0, 2), (0, 3), (1, 2), (1, 3)], [(2, 0), (2, 1), (3, 0), (2, 2), (3, 1), (2, 3), (3, 2)], [(3, 3)]]


In [10]:
groups

[[(0, 0), (0, 1), (1, 1), (1, 0), (0, 2), (0, 3), (1, 2), (1, 3)],
 [(2, 0), (2, 1), (3, 0), (2, 2), (3, 1), (2, 3), (3, 2)],
 [(3, 3)]]

In [11]:
# 가장 짧은 곳을 연결. merge
bridge=[] # (i1,j1,i2,j2,cost)
while len(groups)>1:
    firstgroup = groups[0]
    tmpb = [] 
    mincost=99999
    mincostidx=0
    mincostgroup=0
    for it in firstgroup:
        cost=-1
        # 다른 그룹과 접하는지 검사. lrud
        # 접한다면 비용 기록. (누구랑, 누구랑 연결할 때 비용)
        gi = group_find(it[0], it[1]-1)
        if gi>0:
            cost = abs(input[it[0]][it[1]]-input[it[0]][it[1]-1])
            tmpb.append( (it[0], it[1], it[0], it[1]-1, cost) )
            if mincost>cost:
                mincost=cost
                mincostidx=len(tmpb)-1
                mincostgroup=gi
        gi = group_find(it[0], it[1]+1)
        if gi>0:
            cost = abs(input[it[0]][it[1]]-input[it[0]][it[1]+1])
            tmpb.append( (it[0], it[1], it[0], it[1]+1, cost) )           
            if mincost>cost:
                mincost=cost
                mincostidx=len(tmpb)-1
                mincostgroup=gi
        gi = group_find(it[0]-1, it[1])
        if gi>0:
            cost = abs(input[it[0]][it[1]]-input[it[0]-1][it[1]])
            tmpb.append( (it[0], it[1], it[0]-1, it[1], cost) ) 
            if mincost>cost:
                mincost=cost
                mincostidx=len(tmpb)-1
                mincostgroup=gi
        gi = group_find(it[0]+1, it[1])
        if gi>0:
            cost = abs(input[it[0]][it[1]]-input[it[0]+1][it[1]])
            tmpb.append( (it[0], it[1], it[0]+1, it[1], cost) )           
            if mincost>cost:
                mincost=cost
                mincostidx=len(tmpb)-1
                mincostgroup=gi
    print(tmpb)
    print('cost=',mincost, 'idx=',mincostidx, 'group=', mincostgroup)
    # 최소 비용 캐치. 다리 건설.
    bridge.append(tmpb[mincostidx])
    # 1그룹으로 병합
    group_merge(0, mincostgroup)

print(groups)        
print(bridge)

# total cost
total = 0
for x in bridge:
    total += x[4]
print(total)

[(1, 1, 2, 1, 5), (1, 0, 2, 0, 5), (1, 2, 2, 2, 5), (1, 3, 2, 3, 5)]
cost= 5 idx= 0 group= 1
group merge  1 into 0
[(2, 3, 3, 3, 10), (3, 2, 3, 3, 10)]
cost= 10 idx= 0 group= 1
group merge  1 into 0
[[(0, 0), (0, 1), (1, 1), (1, 0), (0, 2), (0, 3), (1, 2), (1, 3), (2, 0), (2, 1), (3, 0), (2, 2), (3, 1), (2, 3), (3, 2), (3, 3)]]
[(1, 1, 2, 1, 5), (2, 3, 3, 3, 10)]
15


In [291]:
import numpy as np


input=[]
n = 0
# groups=[]
gheight=0
bdebug=False
# bdebug=True

groupmat = None


# gi=group index. -1=new
# return ; group index
def group_add(i,j, gi):
    global groups
    if gi==-1:
        groups.append([(i,j)])
        gi = len(groups)-1
        if bdebug:
            print('newgroup:',gi, 'item=', i,j)
    else:
        groups[gi].append((i,j))
        if bdebug:
            print('add group=',gi, 'item=', i,j)
    return gi

def group_find(i,j):
    if i<0 or j<0:
        return -1
    if i>=n or j>=n:
        return -1
    for gi, g in enumerate(groups):
        if (i,j) in g:
            return gi
    return -1

# d = l, r, u, d  (left, right, up, down)
def can_move(i,j, d):
    i2,j2 = i,j
    if d=='l': j2 -=1
    if d=='r': j2 +=1
    if d=='u': i2 -=1
    if d=='d': i2 +=1
    if i2<0 or j2<0:
        return False
    if i2>=n or j2>=n:
        return False
    return abs(input[i][j] - input[i2][j2])<=gheight

def group_merge(gi, gj):
    global groups
    bak = groups[gj][:]
    if bdebug:
        print('group merge ',gj,'into',gi)
    groups[gi].extend(bak)
    del groups[gj]
    
def group_find2(i,j):
    if i<0 or j<0:
        return -1
    if i>=n or j>=n:
        return -1
    return groupmat[i,j]
    
def solution(land, height):
    global input, n, groups, gheight
    global groupmat
    
    input=land
    gheight = height
    n=len(input)

    print("N=", n)
    groupmat = np.zeros((n,n), dtype=int)
    groupcnt=0
    for i in range(n):
        if bdebug:
            print("progress ", i, "/", n)
            
        for j in range(n):
            gi = groupmat[i,j]
            if gi==0:
                groupcnt+=1
                groupmat[i,j]=groupcnt
                gi = groupcnt

            if can_move(i,j,'r'):
                g2 = groupmat[i,j+1]
                if g2==0:
                    groupmat[i,j+1]=gi
                elif g2==gi:
                    pass
                else:
                    fromg = max(gi,g2)
                    tog = min(gi,g2)
                    groupmat[groupmat==fromg] = tog
                    groupmat[groupmat>fromg] -=1
                    groupcnt-=1
                    gi = tog

            if can_move(i,j,'d'):
                g2 = groupmat[i+1,j]
                if g2==0:
                    groupmat[i+1,j]=gi
                elif g2==gi:
                    pass
                else:
                    fromg = max(gi,g2)
                    tog = min(gi,g2)
                    groupmat[groupmat==fromg] = tog
                    groupmat[groupmat>fromg] -=1
                    groupcnt-=1
                    gi = tog

    if bdebug:
        print('group count=', groupcnt)
        print('groupmat max=', np.max(groupmat))
        print(groupmat)
        
#     return 0 ; # debug phase1
    if bdebug:
        print('phase1 end.')

    # 가장 짧은 곳을 연결. merge
    bridge=[] # (i1,j1,i2,j2,cost)
    allbridge=[]
    scanindex=0
    firstgroup = np.argwhere(groupmat==1).tolist()
    
    while groupcnt>1:
        mincost=99999
        mincostgroup=0
        
        if bdebug:
            print('group count=', groupcnt)
        newcnt = len(firstgroup[scanindex:])
#         if newcnt==0 :
#             print('new incoming data is zero. scanindex=', scanindex)
#       new incomming이 없을 수도 있다. 기존에서 다시 찾아야 함.
# 1그룹에서 다른 그룹으로 연결되는 모든 다리 스캔
        for it in firstgroup[scanindex:]:
            if bdebug:
                print('firstgroupscan ', it)
            cost=-1
            # 다른 그룹과 접하는지 검사. lrud
            # 접한다면 비용 기록. (누구랑, 누구랑 연결할 때 비용)
            gi = group_find2(it[0], it[1]-1)
            if gi>1:
                cost = abs(input[it[0]][it[1]]-input[it[0]][it[1]-1])
                allbridge.append( [it[0], it[1], it[0], it[1]-1, gi, cost])
            gi = group_find2(it[0], it[1]+1)
            if gi>1:
                cost = abs(input[it[0]][it[1]]-input[it[0]][it[1]+1])
                allbridge.append( [it[0], it[1], it[0], it[1]+1, gi, cost])
            gi = group_find2(it[0]-1, it[1])
            if gi>1:
                cost = abs(input[it[0]][it[1]]-input[it[0]-1][it[1]])
                allbridge.append( [it[0], it[1], it[0]-1, it[1], gi, cost])
            gi = group_find2(it[0]+1, it[1])
            if gi>1:
                cost = abs(input[it[0]][it[1]]-input[it[0]+1][it[1]])
                allbridge.append( [it[0], it[1], it[0]+1, it[1], gi, cost])
                
        if bdebug:
            print('allbridge count=', len(allbridge))
        
        # find minimum cost
        tmpb = None
        for x in allbridge:
            if mincost>x[5]:
                mincost=x[5]
                mincostgroup = x[4]
                tmpb = x
        
        if bdebug:
            print('min tmpb=',tmpb)
            print('mincost=',mincost, 'mincostgroup=', mincostgroup)
        
        # mincost 그룹으로 가는 계산은 이제 필요없음. 검색범위에서 제거
        if bdebug:
            print('allbridge=', allbridge)
        allbridge2 = [x for x in allbridge if x[4]!=mincostgroup]
        delbrcount = len(allbridge)-len(allbridge2)
        if bdebug:
            print('allbridge delcount=', delbrcount)
        allbridge = allbridge2
        if bdebug:
            print('allbridge(del)=', allbridge)

        if tmpb==None:
            print('Error!!!: bridge is None??')
            return -1

        # 최소 비용 캐치. 다리 건설.
        bridge.append(tmpb)
        if bdebug:
            print('bridge:', tmpb)

        # 검색 범위 변경 
        scanindex = len(firstgroup)
        if bdebug:
            print('scanindex=', scanindex)

        # 1그룹으로 병합
        mincostindexes = np.argwhere(groupmat==mincostgroup).tolist()
        if bdebug:
            print('mincost indexes(addtofirstgroup)=', mincostindexes)
        firstgroup.extend(mincostindexes)
        if bdebug:
            print('firstgroup count=', len(firstgroup))
            print('')
        
        groupmat[groupmat==mincostgroup] = 1
        groupcnt -=1
#         groupmat[groupmat>mincostgroup] -=1


    if bdebug:
        print(bridge)

    # total cost
    total = 0
    for x in bridge:
        total += x[5]
    if bdebug:
        print('total=', total)
    answer = total
    return answer

In [292]:
import numpy as np

x=np.zeros((5,5), dtype=int)

In [293]:
x[x==2]=22
x

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [294]:
input = [[1, 4, 8, 10], [5, 5, 5, 5], [10, 10, 10, 10], [10, 10, 10, 20]]
height = 3
result=15
solution(input, height)

N= 4


15

In [295]:
for i in range(100):
#     s=np.random.randint(1,100)
#      s=67
    s=79
    print('SEED=',s)
    np.random.seed(s)
    # stress test . max 300x300
    npinput = np.random.rand(5,5)*10
    npinput = np.asarray(npinput, dtype=int)
    print(npinput)
    if solution(npinput.tolist(), 2)==-1:
        break

SEED= 79
[[5 4 5 1 8]
 [3 7 8 0 2]
 [7 8 7 4 5]
 [5 2 1 3 2]
 [2 4 5 0 9]]
N= 5
SEED= 79
[[5 4 5 1 8]
 [3 7 8 0 2]
 [7 8 7 4 5]
 [5 2 1 3 2]
 [2 4 5 0 9]]
N= 5
SEED= 79
[[5 4 5 1 8]
 [3 7 8 0 2]
 [7 8 7 4 5]
 [5 2 1 3 2]
 [2 4 5 0 9]]
N= 5
SEED= 79
[[5 4 5 1 8]
 [3 7 8 0 2]
 [7 8 7 4 5]
 [5 2 1 3 2]
 [2 4 5 0 9]]
N= 5
SEED= 79
[[5 4 5 1 8]
 [3 7 8 0 2]
 [7 8 7 4 5]
 [5 2 1 3 2]
 [2 4 5 0 9]]
N= 5
SEED= 79
[[5 4 5 1 8]
 [3 7 8 0 2]
 [7 8 7 4 5]
 [5 2 1 3 2]
 [2 4 5 0 9]]
N= 5
SEED= 79
[[5 4 5 1 8]
 [3 7 8 0 2]
 [7 8 7 4 5]
 [5 2 1 3 2]
 [2 4 5 0 9]]
N= 5
SEED= 79
[[5 4 5 1 8]
 [3 7 8 0 2]
 [7 8 7 4 5]
 [5 2 1 3 2]
 [2 4 5 0 9]]
N= 5
SEED= 79
[[5 4 5 1 8]
 [3 7 8 0 2]
 [7 8 7 4 5]
 [5 2 1 3 2]
 [2 4 5 0 9]]
N= 5
SEED= 79
[[5 4 5 1 8]
 [3 7 8 0 2]
 [7 8 7 4 5]
 [5 2 1 3 2]
 [2 4 5 0 9]]
N= 5
SEED= 79
[[5 4 5 1 8]
 [3 7 8 0 2]
 [7 8 7 4 5]
 [5 2 1 3 2]
 [2 4 5 0 9]]
N= 5
SEED= 79
[[5 4 5 1 8]
 [3 7 8 0 2]
 [7 8 7 4 5]
 [5 2 1 3 2]
 [2 4 5 0 9]]
N= 5
SEED= 79
[[5 4 5 1 8]
 [3 7 8 0 2]
 [7 8

In [227]:
len(groups)

1

In [ ]:
# max 300

npinput = np.random.rand(300,300)*100
npinput = np.asarray(npinput, dtype=int)
print(npinput)
solution(npinput.tolist(), 20)
    

[[98 31 18 ... 66 44 35]
 [72 74 25 ... 12 88  5]
 [51  0 23 ... 33 10 32]
 ...
 [ 1 23 35 ... 18 41 26]
 [55 15 96 ... 38 62 26]
 [18 59 93 ... 34 17 46]]
N= 300


In [228]:
np.abs(npinput[0]-npinput[0][0])

array([ 0, 17,  6,  0, 12, 10, 11, 35, 42, 16])

In [78]:
np.max(npinput)

99

In [83]:
npinput==23

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [108]:
x2=np.argwhere(npinput==23)[0:5]
print(x2)

[[ 0 13]
 [ 0 72]
 [ 0 85]
 [ 1 34]
 [ 1 85]]


In [109]:
for xi in x2:
    print(xi)

[ 0 13]
[ 0 72]
[ 0 85]
[ 1 34]
[ 1 85]


In [122]:
len(x2)

5

In [124]:
all_bridge=[ [1,1,2,1,1,10],[2,1,2,1,2,10], [3,1,2,1,1,10], [4,1,2,1,1,10],[5,1,2,1,2,10], [6,1,2,1,1,10]]
print(all_bridge)

[[1, 1, 2, 1, 1, 10], [2, 1, 2, 1, 2, 10], [3, 1, 2, 1, 1, 10], [4, 1, 2, 1, 1, 10], [5, 1, 2, 1, 2, 10], [6, 1, 2, 1, 1, 10]]


In [125]:
tmpb=[3,1,2,1,1,10]

In [127]:
# mincost 그룹 다리를 삭제. except tmpb
for x in all_bridge:
    if x[4]==1:
        if x==tmpb:
            pass
        else:
            all_bridge.remove(x)
print(all_bridge)

[[2, 1, 2, 1, 2, 10], [3, 1, 2, 1, 1, 10], [5, 1, 2, 1, 2, 10]]
